In [1]:
import numpy as np
import pandas as pd
import h5py
import sys
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from statistics import mean
import scipy
from pathlib import Path

from radiometric.LUT import create_LUT_df

from NUC.NUC_functions import import_NUC, apply_NUC, get_slice, determine_slope
from NUC.NUC_functions import determine_dark, apply_DFC_to_df
from NUC.NUC_functions import pixel_registration

In [ ]:
#choose filepath to desired data
path = '/Volumes/KESU/may19/LUTS/'
#note cam definitions are flipped as of FLIRPY ffc fix 
cal_file2 =  '/Volumes/KESU/calibration_files/cam1pixel.h5'
cal_file1 =  '/Volumes/KESU/calibration_files/cam2pixel.h5'

In [ ]:
waves= np.linspace(7,12,61)
TEMPS = np.linspace(25,80,12)

In [ ]:
#select ROI
ymin2 = 90; ymax2 = 150;
ymin1 = 120; ymax1 = 180;

FOV = 53.51 ;
HFOV = FOV/4;
angles = np.round(np.linspace(-HFOV,HFOV,ymax2-ymin2))

In [ ]:
'''LOAD slope from 2 point NUC'''
#choose filepath to desired data
save_path =   '/Volumes/KESU/calibration_files/'
name =  "NUC.h5" 

hf = h5py.File(save_path+name, 'r')
M1 = hf.get('M1')
M2 = hf.get('M2')
T1 = hf.get('Tref1')
T2 = hf.get('Tref2')
ROI1 = hf.get('ROI1')
ROI2 = hf.get('ROI2')

In [ ]:
#plot DFCs, check appearance 
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5),sharey = True)
p1 = axes[0].pcolormesh(M1,shading = 'auto')
axes[0].set_title('Camera 1',fontsize = 14)

p2 = axes[1].pcolormesh(M2,shading = 'auto')
axes[1].set_title('Camera 2',fontsize = 14)
cbar = fig.colorbar(p1, ax=axes.ravel().tolist())
cbar.ax.set_ylabel('\n Gain Correction M',fontsize = 12)

plt.show()

In [ ]:
#first, determine reference temp for each camera
ref_name  = '35C.h5'
dark_name = 'dark1.h5'

tref1, tref2 =determine_reference(path,ref_name )

'''determine DFC'''
df_dark = create_LUT_df(path,dark_name)
tref1,tref2,D1,D2 = determine_dark(df_dark,df_ref,M1,M2)

In [ ]:
#apply pixel registration and NUC to full dataset
mean_ims1 =[]
mean_ims2 =[]
std_ims1  =[]
std_ims2  =[]
t1s = []
t2s = []

for i in TEMPS:
    #will create a df and print the names of the keys in the original hdf5 file
    df_i = create_LUT_df(path,str(round(i))+'C.h5')
    
    #apply 2 point NUC, this df cooresponds to Cij
    df = apply_DFC_to_df(df_i,D1,D2,tref1,tref2,M1,M2)
    
    corrected_images1= [] ; 
    corrected_images2= [] ; 
    
    for i in range(len(df)):
        #load image
        rn = df['imgs1'][i];
        
        #apply pixel registration, this returns C lambda, phi
        T  = df['temps1'][i];
        t1s.append(T)
        cn = pixel_registration(rn,cal_file1,waves,ymin1,ymax1)
        corrected_images1.append(cn)
  

    for i in range(len(df)):
        #load image
        rn = df['imgs2'][i];
        
        #apply pixel registration
        T  = df['temps2'][i];
        t2s.append(T)
        cn = pixel_registration(rn,cal_file2,waves,ymin2,ymax2)
        corrected_images2.append(cn)
        

    mean_ims1.append(np.mean(corrected_images1,axis = 0))
    mean_ims2.append(np.mean(corrected_images2,axis = 0))

    std_ims1.append(np.std(corrected_images1,axis = 0))
    std_ims2.append(np.std(corrected_images2,axis = 0))
    
d = {'BB_temps': TEMPS, 'ims1': list(mean_ims1),'s1': list(std_ims1), 'ims2':list(mean_ims2),'s2': list(std_ims2)}
df = pd.DataFrame.from_dict(d, orient='index')
df = df.transpose()

In [ ]:
#create sum column
df["totalrad"] = df['ims2']+df['ims1']
df["totalstd"] = (df['s2']**2+df['s1']**2)**0.5

In [ ]:
#calculate coefficients 
[A1,B1],[A2,B2],[At,Bt] = calc_rad_coef(df)

In [ ]:
'''calculate transmission'''
gamma1,gamma2 = calculate_transmission(df,waves)

In [ ]:
#let's build a lut
temps= np.linspace(0,80,91)
lut1,lut2 = build_LUT(waves, A1,A2,B1,B2)

In [ ]:
#writetohdf5

#choose filepath to desired data
save_path =   '/Volumes/KESU/calibration_files/'
name =  "LUT0519" 

save_LUT(save_path,name,A1,A2,At,B1,B2,Bt,gamma1,gamma2)